# Проект: Анализ базы данных сервиса для чтения книг по подписке

## Цель и задачи

<b>Цель проекта:</b>
    
Провести исследовательский анализ данных сервиса для чтения книг по подписке с целью выявления пользовательских предпочтений, характеристик популярных книг и активности аудитории, чтобы сформировать ценностное предложение для нового продукта.

<b>Задачи:</b>
    
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц для исключения из анализа брошюр;
4. Определить автора с самой высокой средней оценкой книг (учитываются только книги с 50 и более оценками);
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<b>Описание данных:</b>
Для анализа нам предоставлена база данных, содержащая пять таблиц:

1. Таблица <b>books</b>

   Содержит данные о книгах:
   - book_id — идентификатор книги;
   - author_id — идентификатор автора;
   - title — название книги;
   - num_pages — количество страниц;
   - publication_date — дата публикации книги;
   - publisher_id — идентификатор издателя.
      

2. Таблица <b>authors</b>

   Содержит данные об авторах:
   - author_id — идентификатор автора;
   - author — имя автора.
   

3. Таблица <b>publishers</b>

   Содержит данные об издательствах:
   - publisher_id — идентификатор издательства;
   - publisher — название издательства.
   

4. Таблица <b>ratings</b>

   Содержит данные о пользовательских оценках книг:
   - rating_id — идентификатор оценки;
   - book_id — идентификатор книги;
   - username — имя пользователя, оставившего оценку;
   - rating — оценка книги.
   

5. Таблица <b>reviews</b>

   Содержит данные о пользовательских обзорах на книги:
   - review_id — идентификатор обзора;
   - book_id — идентификатор книги;
   - username — имя пользователя, написавшего обзор;
   - text — текст обзора.

<b>План работ:</b>
    
1. Подключение к базе данных
2. Исследование таблиц базы данных
3. Подсчет количества книг, вышедших после 1 января 2000 года
4. Подсчет количества обзоров и средней оценки для каждой книги
5. Определение издательства, которое выпустило наибольшее число книг
6. Определение автора с самой высокой средней оценкой книг
7. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок
8. Вывод и рекомендации

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student',  # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',      # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,                 # порт подключения
    'db': 'data-analyst-final-project-db'  # название базы данных
}

# создаем строку подключения
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode': 'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query: str, engine: sa.engine.base.Engine = engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из SQL, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con=con)

## Исследование таблиц базы данных

Перед тем как приступить к анализу данных, посмортим ближе на ьаблицы датасета.

### Таблица books

In [3]:
# формируем запрос и выводим данные
books_table = '''
SELECT * 
FROM books
LIMIT 5
'''
get_sql_data(books_table)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# формируем запрос и выводим данные
books_table_cnt = '''
SELECT COUNT(book_id) 
FROM books
'''
get_sql_data(books_table_cnt)

,count
0,1000


В таблице `books` записи о 1000 книг, их названиях, количестве страниц, годе выпуска, id издателя и автора книги.

ID издательства и автора понадобятся нам в дальнейшем для связи с таблицами с данными об авторах и издательствах.

### Таблица authors

In [5]:
# формируем запрос и выводим данные
authors_table = '''
SELECT * 
FROM authors
LIMIT 5
'''
get_sql_data(authors_table)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# формируем запрос и выводим данные
authors_table_cnt = '''
SELECT COUNT(author_id) 
FROM authors
'''
get_sql_data(authors_table_cnt)

,count
0,636


В таблице `authors` 636 записей об авторах и всего две колонки с ID и именем автора. Это говорит о том, что некоторым авторам в базе соответствует более одной книги из таблицы `books`. С таблицей `books` они связаны по полю `author_id`.

### Таблица publishers

In [7]:
# формируем запрос и выводим данные
publishers_table = '''
SELECT * 
FROM publishers
LIMIT 5
'''
get_sql_data(publishers_table)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
# формируем запрос и выводим данные
publishers_table_cnt = '''
SELECT COUNT(publisher_id) 
FROM publishers
'''
get_sql_data(publishers_table_cnt)

,count
0,340


В таблице `publishers` 340 записей об авторах и всего две колонки с ID и названием издательства. Это говорит о том, что некоторым издательствам в базе соответствует более одной книги из таблицы `books`. С таблицей `books` они связаны по полю `publisher_id`.

### Таблица ratings

In [9]:
# формируем запрос и выводим данные
ratings_table = '''
SELECT * 
FROM ratings
LIMIT 5
'''
get_sql_data(ratings_table)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
# формируем запрос и выводим данные
ratings_table_cnt = '''
SELECT COUNT(rating_id) 
FROM ratings
'''
get_sql_data(ratings_table_cnt)

,count
0,6456


В таблице `ratings` 6456 записей об оценках книг. Это говорит о том, что некоторым книгам в базе принадлежит более одной оценки. С таблицей `books` оценки связаны по полю `book_id`.

### Таблица reviews

In [11]:
# формируем запрос и выводим данные
reviews_table = '''
SELECT * 
FROM reviews
LIMIT 5
'''
get_sql_data(reviews_table)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
# формируем запрос и выводим данные
reviews_table_cnt = '''
SELECT COUNT(review_id) 
FROM reviews
'''
get_sql_data(reviews_table_cnt)

,count
0,2793


В таблице `reviews` 2793 записей об отзывах о книгах. Это говорит о том, что некоторым книгам в базе принадлежит более одного отзыва. С таблицей `books` отзывы связаны по полю `book_id`.

## Подсчет количества книг, вышедших после 1 января 2000 года

In [13]:
# формируем запрос и выводим данные
books_cnt = '''
SELECT COUNT(book_id) 
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(books_cnt)

,count
0,819


В базе данных <b>819 изданий</b>, вышедших после 1 января 2000 года.

## Подсчет количества обзоров и средней оценки для каждой книги

In [14]:
# формируем запрос и выводим данные
rv_rt_cnt = '''
SELECT b.book_id,
       b.title,
       AVG(rt.rating) AS avg_rating,
       COUNT(rv.review_id) AS reviews_count
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
LEFT JOIN reviews AS rv ON b.book_id=rv.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC, reviews_count DESC
'''
get_sql_data(rv_rt_cnt)

,book_id,title,avg_rating,reviews_count
0,17,A Dirty Job (Grim Reaper #1),5.00,16
1,553,School's Out—Forever (Maximum Ride #2),5.00,12
2,444,Moneyball: The Art of Winning an Unfair Game,5.00,9
3,347,In the Hand of the Goddess (Song of the Liones...,5.00,6
4,610,Tai-Pan (Asian Saga #2),5.00,4
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,12
996,202,Drowning Ruth,2.00,9
997,371,Junky,2.00,4
998,316,His Excellency: George Washington,2.00,4


Средний рейтинг и количество ообзоров книг показывает большую вариативность. Посмотрим на максимальные и минимальные значения среднего рейтинга и количества обзоров. 

In [15]:
# формируем запрос и выводим данные
rv_rt_max_min = '''
WITH avg_rt_cnt_rv AS (
    SELECT b.book_id,
           b.title,
           AVG(rt.rating) AS avg_rating,
           COUNT(rv.review_id) AS reviews_count
    FROM books AS b
    LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
    LEFT JOIN reviews AS rv ON b.book_id=rv.book_id
    GROUP BY b.book_id
)

SELECT MAX(avg_rating) AS max_avg_rating,
       MIN(avg_rating) min_avg_rating,
       MAX(reviews_count) AS max_reviews_count,
       MIN(reviews_count) AS min_reviews_count
FROM avg_rt_cnt_rv
'''
get_sql_data(rv_rt_max_min)

,max_avg_rating,min_avg_rating,max_reviews_count,min_reviews_count
0,5.0,1.5,1120,0


Средний <b>рейтинг</b> книг варьируется <b>от 5 до 1.5</b>, а количество <b>обзоров от 1120 до 0</b>.

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц

Чтобы определить издательство, которое выпустило наибольшее число книг толще
50 страниц, добавим временную таблицу с ID книг соответствующих требованию по количеству страниц.

In [16]:
# формируем запрос и выводим данные
top_publisher = '''
WITH books_only AS (
     SELECT book_id,
            publisher_id
     FROM books
     WHERE num_pages > 50
     )
                   
SELECT p.publisher,
       COUNT(bo.book_id) AS books_count
FROM publishers AS p
RIGHT JOIN books_only AS bo ON p.publisher_id=bo.publisher_id
GROUP BY p.publisher_id
ORDER BY books_count DESC
LIMIT 1
'''
get_sql_data(top_publisher)

,publisher,books_count
0,Penguin Books,42


Наибольшее количество книг (а именнно <b>42 книги</b>) выпустило издательство <b>Penguin Books</b>.

## Определение автора с самой высокой средней оценкой книг

Для определения автора с самой высокой средней оценкой книг отберем только книги с количеством оценок более 50.

In [17]:
# формируем запрос и выводим данные
top_author = '''
WITH books_with_enough_ratings AS (
    SELECT 
        b.book_id,
        b.author_id,
        COUNT(rt.rating_id) AS rating_count,
        AVG(rt.rating) AS avg_book_rating
    FROM books AS b
    JOIN ratings AS rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(rt.rating_id) >= 50),
    
author_rating AS (
    SELECT 
        author_id,
        AVG(avg_book_rating) AS avg_author_rating
    FROM books_with_enough_ratings
    GROUP BY author_id)
    
SELECT 
    a.author,
    ar.avg_author_rating
FROM authors AS a
JOIN author_rating AS ar ON a.author_id = ar.author_id
ORDER BY ar.avg_author_rating DESC
LIMIT 1
'''
get_sql_data(top_author)

,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Наивысшие оценки получают книги авторства <b>J.K. Rowling/Mary GrandPré</b> - средний рейтинг <b>4.28</b> по 5 бальной шкале.

Судя по паре автор/иллюстратор, можно предположить, что наивысшие оценки заработала серия книг о Гарри Поттере.

## Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок

In [18]:
# формируем запрос и выводим данные
users_activity = '''
WITH 
user_ratings_count AS (
    SELECT 
        username,
        COUNT(rating_id) AS ratings_count
    FROM ratings 
    GROUP BY username
    HAVING COUNT(rating_id) > 48
    ),
user_reviews_count AS (
    SELECT 
        username,
        COUNT(review_id) AS reviews_count
    FROM reviews
    WHERE username IN (SELECT username
                       FROM user_ratings_count)
    GROUP BY username
        )

SELECT AVG(reviews_count) AS avg_users_reviews_count
FROM user_reviews_count
'''
get_sql_data(users_activity)

,avg_users_reviews_count
0,24.0


Наиболее вовлеченные пользователи (оставили более 48 оценок к книгам) в среднем оставляют <b>24 обзора</b> на книги в приложении.

## Выводы и рекомендации

В анализируемой базе данных содержатся записи о <b>1000 книг</b>, написанных <b>636 авторами</b> и изданных <b>340 издательствами</b>. Также в базе представлены данные о <b>6456 оценках</b>, формирующих рейтинги книг, и <b>2793 обзора</b>.

В результате анализа были получены следующие выводы:

1. В базе данных <b>819 изданий</b>, вышедших после 1 января 2000 года.

2. Средний рейтинг и количество обзоров книг показывает большую вариативность. <b>Средний рейтинг</b> книг варьируется <b>от 5 до 1.5</b>, а <b>количество обзоров от 1120 до 0</b>.

3. Наибольшее количество книг (а именно <b>42 книги</b>) выпустило издательство <b>Penguin Books</b>.

4. Наивысшие оценки получают книги авторства J.K. Rowling/Mary GrandPré - средний рейтинг 4.28 по 5 бальной шкале. Судя по паре автор/иллюстратор, можно предположить, что наивысшие оценки заработала серия книг о Гарри Поттере.

5. Наиболее вовлеченные пользователи (оставили более 48 оценок к книгам) в среднем оставляют 24 обзора на книги в приложении.
____

<b>Рекомендации:</b>

1. Наивысшие оценки получают популярные серии книг (например, «Гарри Поттер»). Исходя из этого, Рекомендуется провести анализ с целью выявления популярных серий и авторов с высоким рейтингом, отсутствующих в каталоге. Такие серии и книги рекомендуется включить в каталог.

2. Более 80% книг изданы после 2000 года. Можно протестировать интерес пользователей к классической литературе и книгам XX века, добавив их в каталог.

2. С целью повышения вовлеченности и активности пользователей, создания пользовательского контента, рекомендуется продумать и внедрить программу поощрения активных пользователей. Например, разработать систему бэйджей, скидок и\или бонусов.

4. Рекомендуется разработать механизм персонализированных подборок книг для пользователей, оставляющих обзоры и оценки к прочитанным книгам. Это может помочь увеличить вовлеченность и снизить отток пользователей.